<a href="https://colab.research.google.com/github/virf96/Basico/blob/main/Predicci%C3%B3n_de_Despido_de_Colaboradores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dados los datos de los empleados de RRHH, vamos a intentar predecir qué empleados tienen más probabilidades de ser despedidos.

Utilizaremos varios modelos de clasificación para hacer nuestras predicciones.

In [3]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/datasets/HRDataset_v14.csv')

In [12]:
data.head(2)

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,Salary,Termd,PositionID,Position,State,Zip,DOB,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506,0,19,Production Technician I,MA,1960,07/10/83,M,Single,US Citizen,No,White,7/5/2011,NaN,N/A-StillEmployed,Active,Production,Michael Albert,22.0,LinkedIn,Exceeds,4.60,5,0,1/17/2019,0,1
1,"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437,1,27,Sr. DBA,MA,2148,05/05/75,M,Married,US Citizen,No,White,3/30/2015,6/16/2016,career change,Voluntarily Terminated,IT/IS,Simon Roup,4.0,Indeed,Fully Meets,4.96,3,6,2/24/2016,0,17


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Employee_Name               311 non-null    object 
 1   EmpID                       311 non-null    int64  
 2   MarriedID                   311 non-null    int64  
 3   MaritalStatusID             311 non-null    int64  
 4   GenderID                    311 non-null    int64  
 5   EmpStatusID                 311 non-null    int64  
 6   DeptID                      311 non-null    int64  
 7   PerfScoreID                 311 non-null    int64  
 8   FromDiversityJobFairID      311 non-null    int64  
 9   Salary                      311 non-null    int64  
 10  Termd                       311 non-null    int64  
 11  PositionID                  311 non-null    int64  
 12  Position                    311 non-null    object 
 13  State                       311 non

# PREPROCESAMIENTO

## Estandarizar fechas

In [7]:
def encode_dates(df, columns_with_prefixes):
    df = df.copy()
    
    for column, prefix in columns_with_prefixes:
        df[column] = pd.to_datetime(df[column])
        
        df[prefix + '_year'] = df[column].apply(lambda x: x.year)
        df[prefix + '_month'] = df[column].apply(lambda x: x.month)
        df[prefix + '_day'] = df[column].apply(lambda x: x.day)
        
        df = df.drop(column, axis=1)
        
    return df

In [8]:
data_dates = data[['DOB', 'DateofHire', 'LastPerformanceReview_Date']]
data_dates['DOB_datetime'] = pd.to_datetime(data_dates['DOB'])
data_dates['DateofHire_datetime'] = pd.to_datetime(data_dates['DateofHire'])
data_dates['LastPerformanceReview_Date_datetime'] = pd.to_datetime(data_dates['LastPerformanceReview_Date'])
data_dates

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,DOB,DateofHire,LastPerformanceReview_Date,DOB_datetime,DateofHire_datetime,LastPerformanceReview_Date_datetime
0,07/10/83,7/5/2011,1/17/2019,1983-07-10,2011-07-05,2019-01-17
1,05/05/75,3/30/2015,2/24/2016,1975-05-05,2015-03-30,2016-02-24
2,09/19/88,7/5/2011,5/15/2012,1988-09-19,2011-07-05,2012-05-15
3,09/27/88,1/7/2008,1/3/2019,1988-09-27,2008-01-07,2019-01-03
4,09/08/89,7/11/2011,2/1/2016,1989-09-08,2011-07-11,2016-02-01
...,...,...,...,...,...,...
306,05/11/85,7/7/2014,2/28/2019,1985-05-11,2014-07-07,2019-02-28
307,05/04/82,9/2/2008,9/2/2015,1982-05-04,2008-09-02,2015-09-02
308,08/30/79,4/10/2010,2/21/2019,1979-08-30,2010-04-10,2019-02-21
309,02/24/79,3/30/2015,2/1/2019,1979-02-24,2015-03-30,2019-02-01


In [9]:
data_dates['DOB_datetime_year'] = data_dates['DOB_datetime'].apply(lambda x: x.year)
data_dates[['DOB_datetime', 'DOB_datetime_year']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,DOB_datetime,DOB_datetime_year
0,1983-07-10,1983
1,1975-05-05,1975
2,1988-09-19,1988
3,1988-09-27,1988
4,1989-09-08,1989
...,...,...
306,1985-05-11,1985
307,1982-05-04,1982
308,1979-08-30,1979
309,1979-02-24,1979


## Normalización de Variables Dummies

Variables Ordinales

In [10]:
def ordinal_encode(df, columns_with_orderings):
    df = df.copy()
    
    for column, ordering in columns_with_orderings:
        df[column] = df[column].apply(lambda x: ordering.index(x))
        
    return df

Variables Nominales

In [11]:
def onehot_encode(df, columns_with_prefixes):
    df = df.copy()
    
    for column, prefix in columns_with_prefixes:
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    
    return df

Aplicamos el preprocesamiento, normalizamos los datos y realizamos la división de Train y Test

In [13]:
def preprocess_inputs(df, scaler):
    df = df.copy()
    
    # Drop unneeded columns
    df = df.drop(['Employee_Name', 'EmpID', 'MaritalStatusID', 'Sex', 'PositionID', 'DeptID', 'PerfScoreID', 'EmpStatusID', 'EmploymentStatus', 'DateofTermination', 'TermReason', 'ManagerID'], axis=1)
    ''
    # Date encoding
    date_columns = [
        ('DOB', "DOB"),
        ('DateofHire', "DOH"),
        ('LastPerformanceReview_Date', "PRD")
    ]
    df = encode_dates(df, columns_with_prefixes=date_columns)
    
    # Ordinal encoding
    ordinal_columns = [
        ('PerformanceScore', ['PIP', 'Needs Improvement', 'Fully Meets', 'Exceeds'])
    ]
    df = ordinal_encode(df, columns_with_orderings=ordinal_columns)
    
    # One-hot encoding
    nominal_columns = [
        ('Position', "PS"),
        ('State', "ST"),
        ('Zip', "ZP"),
        ('MaritalDesc', "MD"),
        ('CitizenDesc', "CD"),
        ('HispanicLatino', "HL"),
        ('RaceDesc', "RD"),
        ('Department', "DE"),
        ('ManagerName', "MN"),
        ('RecruitmentSource', "RS")
    ]
    df = onehot_encode(df, columns_with_prefixes=nominal_columns)
    
    # Split df into X and y
    y = df['Termd'].copy()
    X = df.drop('Termd', axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)
    
    # Scale X
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [14]:
scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = preprocess_inputs(data, scaler)

In [15]:
X_train

,MarriedID,GenderID,FromDiversityJobFairID,Salary,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,DaysLateLast30,Absences,DOB_year,DOB_month,DOB_day,DOH_year,DOH_month,DOH_day,PRD_year,PRD_month,PRD_day,PS_Accountant I,PS_Administrative Assistant,PS_Area Sales Manager,PS_BI Developer,PS_BI Director,PS_CIO,PS_Data Analyst,PS_Data Analyst,PS_Data Architect,PS_Database Administrator,PS_Director of Operations,PS_Director of Sales,PS_Enterprise Architect,PS_IT Director,PS_IT Manager - DB,PS_IT Manager - Infra,PS_IT Manager - Support,PS_IT Support,PS_Network Engineer,PS_President & CEO,PS_Principal Data Architect,...,RD_Black or African American,RD_Hispanic,RD_Two or more races,RD_White,DE_Admin Offices,DE_Executive Office,DE_IT/IS,DE_Production,DE_Sales,DE_Software Engineering,MN_Alex Sweetwater,MN_Amy Dunn,MN_Board of Directors,MN_Brandon R. LeBlanc,MN_Brannon Miller,MN_Brian Champaigne,MN_David Stanley,MN_Debra Houlihan,MN_Elijiah Gray,MN_Eric Dougall,MN_Janet King,MN_Jennifer Zamora,MN_John Smith,MN_Kelley Spirea,MN_Ketsia Liebig,MN_Kissy Sullivan,MN_Lynn Daneault,MN_Michael Albert,MN_Peter Monroe,MN_Simon Roup,MN_Webster Butler,RS_CareerBuilder,RS_Diversity Job Fair,RS_Employee Referral,RS_Google Search,RS_Indeed,RS_LinkedIn,RS_On-line Web application,RS_Other,RS_Website
0,0.0,1.0,0.0,0.080510,0.000000,0.208763,0.50,0.000,1.000000,0.210526,0.040404,0.909091,0.033333,0.666667,0.727273,0.933333,1.000000,0.000,0.966667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.048133,0.333333,0.734536,0.75,0.000,0.500000,0.736842,0.030303,0.636364,0.266667,0.500000,0.272727,0.033333,0.555556,0.000,0.633333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,1.0,0.270076,0.666667,0.896907,0.50,0.500,0.000000,0.421053,0.121212,0.454545,0.433333,0.583333,0.090909,0.566667,0.777778,0.125,0.366667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.083409,0.666667,0.548969,1.00,0.000,0.000000,0.736842,0.060606,0.636364,0.600000,0.416667,0.272727,0.100000,1.000000,0.000,0.433333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.051281,0.666667,0.793814,1.00,0.000,0.000000,0.631579,0.010101,0.545455,0.066667,0.416667,0.545455,0.133333,0.222222,0.000,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,0.0,0.0,0.0,0.082916,0.666667,0.639175,1.00,0.000,0.000000,0.157895,0.131313,0.545455,0.200000,0.583333,0.545455,0.233333,1.000000,0.000,0.033333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [16]:
y_train

67     0
105    1
285    1
34     0
234    1
      ..
203    0
255    0
72     0
235    1
37     0
Name: Termd, Length: 217, dtype: int64

# Modelos

Training

In [17]:
models = {
    "   K-Nearest Neighbors": KNeighborsClassifier(),
    "   Logistic Regression": LogisticRegression(),
    "Support Vector Machine": SVC(),
    "         Decision Tree": DecisionTreeClassifier(),
    "        Neural Network": MLPClassifier()
}

for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

   K-Nearest Neighbors trained.
   Logistic Regression trained.
Support Vector Machine trained.
         Decision Tree trained.
        Neural Network trained.


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Results

In [18]:
for name, model in models.items():
    print(name + " Accuracy: {:.2f}%".format(model.score(X_test, y_test) * 100))

   K-Nearest Neighbors Accuracy: 76.60%
   Logistic Regression Accuracy: 91.49%
Support Vector Machine Accuracy: 90.43%
         Decision Tree Accuracy: 98.94%
        Neural Network Accuracy: 80.85%
